In [1]:
import os

In [2]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\karth\\ML_practice\\Projects\\EndToEnd_MLOps_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.ML_Project.constants import *
from src.ML_Project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion
          create_directories([config.root_dir])         

          data_ingestion_config = DataIngestionConfig(
                root_dir= config.root_dir,
                source_URL= config.source_URL,
                local_data_file= config.local_data_file,
                unzip_dir= config.unzip_dir 
          )
          return data_ingestion_config   

In [8]:
from urllib import request 
import zipfile
from src.ML_Project import logger
from src.ML_Project.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-23 14:45:35,036: INFO: common: Yaml file:config\config.yaml loaded successfully]
[2025-03-23 14:45:35,040: INFO: common: Yaml file:params.yaml loaded successfully]
[2025-03-23 14:45:35,045: INFO: common: Yaml file:schema.yaml loaded successfully]
[2025-03-23 14:45:35,049: INFO: common: created directory at: artifacts]
[2025-03-23 14:45:35,051: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-23 14:45:37,617: INFO: 3627867303: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 34106
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c01798b54aab61bf15824f2182485b623eae9f05e0aebd6115058880b8ae521c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: CB48:17B82:1029A6:2DF48D:67DFD138
Accept-Ranges: bytes
Date: Sun, 23 Mar